# SB1 Big Data Downloads
An analysis into the corridor analysis data downloads from Streetlight and Replica

In [1]:
import pandas as pd
from siuba import *

from calitp_data_analysis.sql import to_snakecase

import altair as alt
from calitp_data_analysis import calitp_color_palette as cp

from IPython.display import Markdown, HTML, display_html, display
from IPython.core.display import display

import numpy as np

/tmp/ipykernel_306/4150955979.py:10: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display


In [126]:
# ! pip install nltk

In [3]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import re

In [4]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 100)

In [5]:
GCS_PATH = "gs://calitp-analytics-data/data-analyses/sb125/corridor_study_data/"

### Read in Replica Data Spring 2023

In [6]:
replica_trips = "replica-bay_bridge-03_06_24-trips_dataset.csv"

In [7]:
replica_ppl = "replica-bay_bridge-03_06_24-people_dataset.csv"

In [8]:
r_trips = to_snakecase(pd.read_csv(f"{GCS_PATH}{replica_trips}"))

/tmp/ipykernel_306/2643890524.py:1: DtypeWarning: Columns (2,4,5,9,10,11,18,25,26,29,30,35,36,40,43,48,50) have mixed types. Specify dtype option on import or set low_memory=False.
  r_trips = to_snakecase(pd.read_csv(f"{GCS_PATH}{replica_trips}"))


In [9]:
display(HTML("<h4><strong> Sample of data</strong></h4>"))


In [10]:
r_trips.sample(5)

,origin_building_use,trip_duration_minutes,trip_taker_language,origin_trct_2020,trip_taker_household_id,trip_taker_sex,trip_taker_home_bgrp_2020,origin_st_2020,trip_taker_home_st_2020,trip_taker_race_ethnicity,transit_agency,transit_route,trip_taker_resident_type,vehicle_type,trip_taker_age,activity_id,destination_bgrp_2020,destination_building_use,trip_taker_wfh,destination_st_2020,destination_cty_2020,trip_taker_work_bgrp_2020,destination_trct_2020,trip_taker_household_income,origin_bgrp_2020,trip_taker_commute_mode,trip_taker_available_vehicles,primary_mode,previous_trip_purpose,trip_taker_building_type,transit_submode,trip_taker_work_st_2020,trip_taker_household_size,origin_cty_2020,destination_land_use,trip_taker_industry,trip_taker_tenure,vehicle_fuel_type,trip_taker_home_trct_2020,trip_taker_work_trct_2020,trip_taker_education,trip_start_time,trip_taker_individual_income,trip_taker_employment_status,trip_purpose,origin_land_use,trip_taker_work_cty_2020,trip_distance_miles,trip_taker_school_grade_attending,trip_taker_home_cty_2020,trip_taker_person_id,trip_end_time
344797,retail,139,NaN,"206.05 (Placer, CA)",,NaN,Visitor (no home location),California,Visitor (no home location),NaN,NaN,NaN,visitor,unknown_vehicle_type,NaN,8758582875719778070,"1 (Tract 9843, San Mateo, CA)",transportation_utilities,NaN,California,San Mateo,Visitor (no work/school location),"9843 (San Mateo, CA)",NaN,"3 (Tract 206.05, Placer, CA)",NaN,NaN,private_auto,eat,NaN,NaN,Visitor (no work/school location),NaN,Placer,transportation_utilities,NaN,NaN,unknown_fuel_type,Visitor (no home location),Visitor (no work/school location),NaN,21:23:00,NaN,NaN,region_departure,retail,Visitor (no work/school location),117.9,NaN,Visitor (no home location),6466724364837963852,23:42:15
404006,retail,39,spanish,"9809 (San Francisco, CA)",5717162922661989573,female,"2 (Tract 2506.01, Solano, CA)",California,California,hispanic_or_latino_origin,NaN,NaN,core,unknown_vehicle_type,46.0,9780660917558834164,"2 (Tract 2506.01, Solano, CA)",single_family,in_person,California,Solano,"1 (Tract 9809, San Francisco, CA)","2506.01 (Solano, CA)",26234.0,"1 (Tract 9809, San Francisco, CA)",private_auto,two,auto_passenger,work,single_family,NaN,California,1.0,San Francisco,single_family,naics44_45,renter,unknown_fuel_type,"2506.01 (Solano, CA)","9809 (San Francisco, CA)",high_school,10:15:00,26234.0,employed,other_activity_type,mixed_use,San Francisco,30.3,not_attending_school,Solano,17623584878492347501,10:54:09
47514,single_family,40,english,"4305 (Alameda, CA)",9450366543874170043,female,"2 (Tract 4305, Alameda, CA)",California,California,white_not_hispanic_or_latino,NaN,NaN,core,unknown_vehicle_type,29.0,17995626605244124712,"2 (Tract 607.01, San Francisco, CA)",retail,in_person,California,San Francisco,"2 (Tract 607.01, San Francisco, CA)","607.01 (San Francisco, CA)",177915.0,"2 (Tract 4305, Alameda, CA)",private_auto,two,private_auto,home,single_family,NaN,California,2.0,Alameda,retail,naics712130,owner,other_non_bev,"4305 (Alameda, CA)","607.01 (San Francisco, CA)",bachelors_degree,05:52:00,31293.0,employed,work,single_family,San Francisco,25.3,not_attending_school,Alameda,6921358500856400741,06:32:50
386134,retail,27,asian_pacific,"607.01 (San Francisco, CA)",14736001622789934895,female,"2 (Tract 4445, Alameda, CA)",California,California,asian_not_hispanic_or_latino,NaN,NaN,core,unknown_vehicle_type,44.0,15348858204039127111,"4 (Tract 4003, Alameda, CA)",retail,in_person,California,Alameda,"4 (Tract 4003, Alameda, CA)","4003 (Alameda, CA)",162728.0,"3 (Tract 607.01, San Francisco, CA)",private_auto,three_plus,auto_passenger,shop,single_family,NaN,California,4.0,San Francisco,retail,naics722511,owner,unknown_fuel_type,"4445 (Alameda, CA)","4003 (Alameda, CA)",high_school,10:01:00,61795.0,employed,work,retail,Alameda,12.2,not_attending_school,Alameda,9306277843367464539,10:28:22
318313,retail,43,english,"3451.03 (Contra Costa, CA)",10415214486910735210,male,"1 (Tract 176.04, Sa

In [11]:
display(HTML("<h4><strong> Columns in Replica Trips Data</strong></h4>"))


In [12]:
r_trips.columns

Index(['origin_building_use', 'trip_duration_minutes', 'trip_taker_language',
       'origin_trct_2020', 'trip_taker_household_id', 'trip_taker_sex',
       'trip_taker_home_bgrp_2020', 'origin_st_2020',
       'trip_taker_home_st_2020', 'trip_taker_race_ethnicity',
       'transit_agency', 'transit_route', 'trip_taker_resident_type',
       'vehicle_type', 'trip_taker_age', 'activity_id',
       'destination_bgrp_2020', 'destination_building_use', 'trip_taker_wfh',
       'destination_st_2020', 'destination_cty_2020',
       'trip_taker_work_bgrp_2020', 'destination_trct_2020',
       'trip_taker_household_income', 'origin_bgrp_2020',
       'trip_taker_commute_mode', 'trip_taker_available_vehicles',
       'primary_mode', 'previous_trip_purpose', 'trip_taker_building_type',
       'transit_submode', 'trip_taker_work_st_2020',
       'trip_taker_household_size', 'origin_cty_2020', 'destination_land_use',
       'trip_taker_industry', 'trip_taker_tenure', 'vehicle_fuel_type',
       't

#### Data Exploration: Replica Trips
* Summarizing the data that we exported from Replica
* Existing visualizations in replica: 
   * Primary Mode
   * Trip Purpose
   * Starting hour 
   * Trip Duration/Distance
   * Origin Destination
   * Vehicle Fuel type
   * Transit Routes/Stops/Sub mode/Agency
   * Household Income
   * Race and Ethnicity
   * Private Auto Availability
   * Age
   * Employment/School Status
   
   

In [13]:
trips = r_trips>>group_by(_.primary_mode)>>summarize(avg_trip_time = _.trip_duration_minutes.mean(),
                                                     number_trips = _.activity_id.nunique())

In [14]:
chart = (
        alt.Chart(trips)
        .mark_bar()
        .encode(
            x=alt.X("primary_mode"),
            y=alt.Y("avg_trip_time"),
            color=alt.Color("avg_trip_time", scale=alt.Scale(range = cp.CALITP_DIVERGING_COLORS)
            ), tooltip=trips.columns.tolist())
      .properties(title = "Average Trip Time",
    width=800,
    height=300)
        )
chart 

alt.Chart(...)

In [15]:
chart = (
        alt.Chart(trips)
        .mark_bar()
        .encode(
            x=alt.X("primary_mode"),
            y=alt.Y("number_trips"),
            color=alt.Color("number_trips", scale=alt.Scale(range = cp.CALITP_DIVERGING_COLORS,)
            ), tooltip=trips.columns.tolist())
      .properties(title="Number of Trips",
    width=800,
    height=300)
        )
chart 

alt.Chart(...)

In [16]:
display(HTML("<h4><strong>Length of Trips dataframe</strong></h4>"))

In [17]:
len(r_trips)

674864

In [18]:
display(HTML("<h4><strong>Number of unique activity ids in data</strong></h4>"))

In [19]:
r_trips>>summarize(number_trips = _.activity_id.nunique())

,number_trips
0,674864


In [20]:
display(HTML("<h4><strong>Number of unique trip taker ids</strong></h4>")) 

In [21]:
r_trips>>summarize(number_trips = _.trip_taker_person_id.nunique())

,number_trips
0,309596


In [22]:
race_ethnicity_by_mode = r_trips>>group_by(_.primary_mode)>>count(_.trip_taker_race_ethnicity)

In [23]:
display(HTML("<h4><strong>Trip Taker Race and Ethnicity by Mode</strong></h4>")) 

In [24]:
chart = (
        alt.Chart(race_ethnicity_by_mode)
        .mark_bar()
        .encode(
            x=alt.X("primary_mode"),
            y=alt.Y("n"),
            color=alt.Color("trip_taker_race_ethnicity", scale=alt.Scale(range = cp.CALITP_DIVERGING_COLORS,
                                                                         domain=race_ethnicity_by_mode["trip_taker_race_ethnicity"].unique().tolist())
            ),
            tooltip=race_ethnicity_by_mode.columns.tolist())
            
      .properties(
    width=800,
    height=300)
        )
chart 

alt.Chart(...)

In [25]:
display(HTML("<H4><strong>Transit Mode Splits</strong></H4>")) 


In [26]:
display(HTML("<strong>Looking at transit mode, need to alter the transit_agency col and transit_submode to get a better count.</strong>")) 


In [27]:
ptt_agency_count = (r_trips
 >>filter(_.primary_mode=="public_transit")
 >>group_by(_.primary_mode, _.transit_submode, _.transit_agency)
 >>summarize(n =_.activity_id.nunique())
 >>arrange(-_.n))

In [129]:
# ptt_agency_count

In [29]:
display(HTML("<h4><strong>Most common transit mode combinations</strong></h4>")) 


In [30]:
ptt_modes = (r_trips
             >>filter(_.primary_mode =="public_transit")
             >>count(_.transit_submode)>>arrange(-_.n))

In [31]:
ptt_modes.head(10)

,transit_submode,n
0,bus,2999
1,"bus, bus",1706
21,"bus, light_rail",316
2,"bus, bus, bus",302
41,"light_rail, bus, bus",127
40,"light_rail, bus",76
55,"subway, bus",66
32,"bus, subway",63
14,"bus, bus, light_rail",49
33,"bus, subway, bus",46


In [32]:
##### unnesting the transit submode and agencies to get counts. 

In [33]:
ptt_agency_count['transit_submode'] = ptt_agency_count['transit_submode'].astype(str)
ptt_agency_count['transit_agency'] = ptt_agency_count['transit_agency'].astype(str)

In [34]:
ptt_agency_count['n_modes_taken'] = ptt_agency_count.transit_submode.apply(lambda x: len(x.split()))

In [35]:
# ptt_agency_count>>arrange(-_.n_modes_taken)

In [36]:
## This line of code gets all agencies listed, even if agency is named twice. 
## Need code that counts the unique agencies
#ptt_agency_count['agency_count'] = ptt_agency_count.transit_agency.apply(lambda x: len(x.split(", ")))

In [37]:
ptt_agency_count['agency_count'] = [len(set(x.split(", "))) for x in
                        ptt_agency_count['transit_agency'].str.lower()]

In [38]:
modes_count = ptt_agency_count>>group_by(_.n_modes_taken)>>summarize(n_trips = _.n.sum())

In [130]:
# modes_count

In [40]:
chart = (
        alt.Chart((modes_count))
        .mark_bar(size=60)
        .encode(
            x=alt.X("n_modes_taken", title ="Number of Modes Taken per Trip"),
            y=alt.Y("n_trips", title = "Number of Trips"),
            color=alt.Color("n_trips", title = "Number of Trips",
                            scale=alt.Scale(range = cp.CALITP_DIVERGING_COLORS)),
            tooltip=modes_count.columns.tolist())
            
      .properties(title = "How Many Modes are Taken Per Trip",
    width=800,
    height=300)
        )
chart 

alt.Chart(...)

In [41]:
# ptt_agency_count>>group_by(_.agency_count)>>count(_.n_modes_taken)>>arrange(-_.n)
agency_mode_trips = ptt_agency_count>>group_by(_.agency_count, _.n_modes_taken)>>summarize(ntrips=_.n.sum())>>arrange(-_.ntrips)

In [42]:
chart = (
        alt.Chart((agency_mode_trips))
        .mark_circle(size=100)
        .encode(
            x=alt.X("agency_count", title ="Number of Agencies"),
            y=alt.Y("n_modes_taken", title = "Number of Modes Taken"),
            color=alt.Color("ntrips", title = "Number of Trips",
                            scale=alt.Scale(range = cp.CALITP_DIVERGING_COLORS)),
            tooltip=agency_mode_trips.columns.tolist())
            
      .properties(title = "How Many Modes are Taken Per Trip",
    width=800,
    height=300)
        )
chart 

alt.Chart(...)

In [43]:
##### Getting columns for each agency and counts 

In [44]:
# ptt_agency_count['transit_agency'].unique().tolist()

In [45]:
## Get just one columns
column = ptt_agency_count[["transit_agency"]]
#remove single-dimensional entries from the shape of an array
col_text = column.squeeze()
# get list of words
text_list = col_text.tolist()
# #join list of words 
text_list = ', '.join(text_list).title()

In [46]:
text_list = text_list.replace(", ", "', '")

In [47]:
text_list = "['" + text_list + "']"

In [48]:
# text_list

In [49]:
import ast


In [50]:
agency_list = ast.literal_eval(text_list)

In [51]:
agency_list = set(agency_list)

In [52]:
# len(agency_list)

In [53]:
display(HTML("<h4><strong>Agencies Identified in Trips Data</strong></h4>")) 

(agency_list)

{'Ac Transit',
 'Amtrak',
 'Bay Area Rapid Transit',
 'Caltrain',
 'Fairfield And Suisun Transit',
 'Marin Transit',
 'Nan',
 'Sacramento Regional Transit',
 'Samtrans',
 'San Francisco Bay Ferry',
 'San Francisco Municipal Transportation Agency',
 'San Joaquin Regional Transit District (Rtd)',
 'Santa Cruz Metro',
 'Soltrans',
 'The S',
 'Tri Delta Transit',
 'Vta'}

In [54]:
#### using this stackoverflow code to count the instances of the transit agencies in the data

In [55]:
## https://stackoverflow.com/questions/46250505/count-the-number-of-times-multiple-substrings-appear-in-dataframe-column

In [56]:
# ptt_agency_count['transit_agency'] = ptt_agency_count['transit_agency'].str.strip('()').str.split(',')

In [57]:
# ptt_agency_count.join(ptt_agency_count['transit_agency'].str.join('|').str.get_dummies())

In [58]:
### making a copy of the ptt_agency_count
df = ptt_agency_count.copy()

In [59]:
# df['transit_agency'] = df['transit_agency'].astype(str)

In [60]:
# df2 = df['transit_agency'].str.get_dummies(sep=', ')

In [61]:
# df2

##### Trying ChatGPT approach

In [62]:
agency_indv_count = (r_trips>>filter(_.primary_mode=="public_transit")>>select(_.transit_agency))
agency_indv_count['transit_agency'] = agency_indv_count['transit_agency'].astype(str)

In [63]:
# transit_agencies = set()
# for agencies in df['transit_agency'].str.split(', '):
#     transit_agencies.update(agencies)

In [64]:
# unique_agencies = []

In [65]:
# for agency in transit_agencies:
#     df[agency] = df['transit_agency'].str.count(agency)
#     unique_agencies.append(agency)

In [66]:
# ### adding column for unique agencies list
# def get_unique_agencies(agency_list):
#     unique_agencies = set()
#     for agencies in agency_list:
#         unique_agencies.update(agencies.split(', '))
#     return ', '.join(sorted(list(unique_agencies)))

# # Applying the function to each row of the dataframe to get unique agencies
# df['unique_agencies'] = df['transit_agency'].str.split(', ').apply(lambda x: get_unique_agencies(x))

In [67]:
def get_dummies_by_agency(df, col):
    transit_agencies = set()
    for agencies in df[col].str.split(', '):
        transit_agencies.update(agencies)
    unique_agencies = []
    
    for agency in transit_agencies:
        df[agency] = df[col].str.count(agency)
        unique_agencies.append(agency)

    ### adding column for unique agencies list
    def get_unique_agencies(agency_list):
        unique_agencies = set()
        for agencies in agency_list:
            unique_agencies.update(agencies.split(', '))
        return ', '.join(sorted(list(unique_agencies)))

    # Applying the function to each row of the dataframe to get unique agencies
    df['unique_agencies'] = df[col].str.split(', ').apply(lambda x: get_unique_agencies(x))
    
    return df

In [68]:
df = get_dummies_by_agency(df, "transit_agency")

In [69]:
agency_indv_count = get_dummies_by_agency(agency_indv_count, "transit_agency")

In [70]:
##### Identifying trips with one agency

In [71]:
cols_to_keep = ["transit_submode","unique_agencies", "n","n_modes_taken"]

In [72]:
# chart = (alt.Chart(df)
#              .mark_circle(size=100)
#              .encode(
#                  x=alt.X("n_modes_taken", title="Number of Modes taken"),
#                  y=alt.Y("n", title="Number of Trips"),
#                  color = alt.Color("agency_count", title="Number of Unique Agencies",
#                                   scale=alt.Scale(
#                                       range=cp.CALITP_DIVERGING_COLORS,
#                                       domain=df["agency_count"].unique().tolist())
#             ),
#             tooltip=cols_to_keep)
#         .properties(title = ("Transit Trips Agency Breakdown"), width=500,
#     height=300)
#         )
# chart                    

In [73]:
# (df>>filter(_.agency_count==1)>>arrange(-_.n))

In [74]:
chart = (alt.Chart((df>>filter(_.agency_count==1)))
             .mark_bar()
             .encode(
                 x=alt.X("unique_agencies", title="Agency"),
                 y=alt.Y("n", title="Number of Modes taken"),
                 color = alt.Color("n_modes_taken", title="Number of Trips",
                                  scale=alt.Scale(
                                      range=cp.CALITP_SEQUENTIAL_COLORS,)),
            tooltip=cols_to_keep)
        .properties(title = ("Transit Trips With Only One Agency"), width=800,
    height=500)
        )
chart           

alt.Chart(...)

In [75]:
display(HTML("<h4><strong>Most Common Agency Combination</strong></h4>")) 

(df>>arrange(-_.n)>>select(_.transit_agency)).head(5)

,transit_agency
0,AC TRANSIT
12,"San Francisco Municipal Transportation Agency, AC TRANSIT"
8,"AC TRANSIT, San Francisco Municipal Transportation Agency"
6,"AC TRANSIT, AC TRANSIT"
75,"AC TRANSIT, San Francisco Municipal Transportation Agency"


In [76]:
columns_to_keep = ["AC TRANSIT",
                   "San Francisco Bay Ferry",
                   "SolTrans",
                   "Caltrain",
                   "SamTrans",
                   "The S",
                   "Fairfield and Suisun Transit",
                   "Sacramento Regional Transit",
                   "Marin Transit",
                   "Tri Delta Transit",
                   "San Francisco Municipal Transportation Agency",
                   "Amtrak",
                   "San Joaquin Regional Transit District (RTD)",
                   "Bay Area Rapid Transit",
                   "Santa Cruz Metro",
                   "VTA"
                  ]

In [124]:
# agency_indv_count.sample()

In [78]:
df_agencies = agency_indv_count.loc[:, columns_to_keep]

In [79]:
df_agencies = pd.DataFrame(df_agencies).transpose().reset_index().rename(columns={'index':'agency'})

In [123]:
# df_agencies

In [81]:
df_agencies['n_trips'] = df_agencies[list(df_agencies.columns)].sum(axis=1)

/tmp/ipykernel_306/2517619377.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_agencies['n_trips'] = df_agencies[list(df_agencies.columns)].sum(axis=1)


In [82]:
df_agencies = df_agencies>>select(_.agency, _.n_trips)

In [83]:
tooltip_cols = ["agency", "n_trips"]

In [131]:
# df_agencies

In [85]:
chart = (alt.Chart(df_agencies)
        .mark_bar()
        .encode(
            x=alt.X("agency", title = "Agency Name"),
            y=alt.Y("n_trips", title= "Number of boardings reported for trips (One person taking two AC Transit trips will count as 2)"),
            color=alt.Color("n_trips", scale=alt.Scale(range = cp.CALITP_SEQUENTIAL_COLORS)),
            tooltip = tooltip_cols)
      .properties(title = "Number of Times an Agency was used for Trip taking over Bay Bridge",
    width=800,
    height=500)
        )
chart 

alt.Chart(...)

In [86]:
display(HTML("<h4><strong>Trips by Resident Type</strong></h4>")) 


In [87]:
trip_by_res_type = (r_trips
 >>group_by(_.primary_mode,_.trip_taker_resident_type)
 >>summarize(number_trips = _.activity_id.nunique())
 >>arrange(_.primary_mode))

In [125]:
trip_by_res_type

,primary_mode,trip_taker_resident_type,number_trips
0,auto_passenger,core,367739
1,auto_passenger,visitor,23552
2,commercial,NaN,11897
3,on_demand_auto,core,24372
4,on_demand_auto,visitor,2581
5,private_auto,core,232486
6,private_auto,visitor,6204
7,public_transit,core,5986
8,public_transit,visitor,47


In [159]:
chart = (
        alt.Chart(trip_by_res_type)
        .mark_bar()
        .encode(
            x=alt.X("primary_mode", title = "Mode"),
            y=alt.Y("number_trips", title = "Number of Trips"),
            color=alt.Color("trip_taker_resident_type", scale=alt.Scale(range = cp.CALITP_DIVERGING_COLORS,
                                                                         domain=trip_by_res_type["trip_taker_resident_type"].unique().tolist())
            ),
            tooltip=trip_by_res_type.columns.tolist())
      .properties(title = "Trips by Resident Type",
    width=800,
    height=500)
        )
chart 

alt.Chart(...)

In [90]:
display(HTML("<h4><strong>Unique Household Ids</strong></h4>")) 

In [91]:
r_trips>>count(_.trip_taker_household_id)>>arrange(-_.n)

,trip_taker_household_id,n
248686,,27897
262322,NaN,16384
110604,8194219563640751815,28
82299,6097771312917788596,27
74914,5552467881171814730,22
...,...,...
262312,9988489366890220188,1
262316,9991342309439330560,1
262317,9992490903429866665,1
262318,9992599661009943006,1


In [92]:
display(HTML("<h4><strong>Checking one household id</strong></h4>")) 


In [93]:
print(len(r_trips>>filter(_.trip_taker_household_id == 8194219563640751815)))
(r_trips>>filter(_.trip_taker_household_id == 8194219563640751815)).sample()

28


,origin_building_use,trip_duration_minutes,trip_taker_language,origin_trct_2020,trip_taker_household_id,trip_taker_sex,trip_taker_home_bgrp_2020,origin_st_2020,trip_taker_home_st_2020,trip_taker_race_ethnicity,transit_agency,transit_route,trip_taker_resident_type,vehicle_type,trip_taker_age,activity_id,destination_bgrp_2020,destination_building_use,trip_taker_wfh,destination_st_2020,destination_cty_2020,trip_taker_work_bgrp_2020,destination_trct_2020,trip_taker_household_income,origin_bgrp_2020,trip_taker_commute_mode,trip_taker_available_vehicles,primary_mode,previous_trip_purpose,trip_taker_building_type,transit_submode,trip_taker_work_st_2020,trip_taker_household_size,origin_cty_2020,destination_land_use,trip_taker_industry,trip_taker_tenure,vehicle_fuel_type,trip_taker_home_trct_2020,trip_taker_work_trct_2020,trip_taker_education,trip_start_time,trip_taker_individual_income,trip_taker_employment_status,trip_purpose,origin_land_use,trip_taker_work_cty_2020,trip_distance_miles,trip_taker_school_grade_attending,trip_taker_home_cty_2020,trip_taker_person_id,trip_end_time
594972,retail,6,indo_european,"615.06 (San Francisco, CA)",8194219563640751815,male,"1 (Tract 179.03, San Francisco, CA)",California,California,white_not_hispanic_or_latino,NaN,NaN,core,unknown_vehicle_type,27.0,1397390795811450502,"1 (Tract 179.03, San Francisco, CA)",single_family,remote,California,San Francisco,"1 (Tract 179.03, San Francisco, CA)","179.03 (San Francisco, CA)",1200990.0,"3 (Tract 615.06, San Francisco, CA)",worked_from_home,three_plus,auto_passenger,shop,single_family,NaN,California,10.0,San Francisco,single_family,naics54,renter,unknown_fuel_type,"179.03 (San Francisco, CA)","179.03 (San Francisco, CA)",bachelors_degree,11:24:00,218735.0,employed,home,mixed_use,San Francisco,3.7,not_attending_school,San Francisco,14409692340574959811,11:30:46


In [94]:
## checking one household id
r_trips>>filter(_.trip_taker_household_id == 8194219563640751815)>>count(_.trip_taker_person_id, _.trip_taker_age,
                                                                         _.trip_taker_sex, _.trip_taker_household_size)

,trip_taker_person_id,trip_taker_age,trip_taker_sex,trip_taker_household_size,n
0,1325181746552868554,22.0,female,10.0,6
1,2310331066900902679,30.0,female,10.0,6
2,4746831412975349070,34.0,female,10.0,2
3,10066255028734967962,28.0,male,10.0,4
4,11058477761487230854,32.0,male,10.0,2
5,12894946300265742193,39.0,male,10.0,2
6,14409692340574959811,27.0,male,10.0,6


#### Adding in Replica's People Dataset

In [95]:
r_ppl = to_snakecase(pd.read_csv(f"{GCS_PATH}{replica_ppl}"))

/tmp/ipykernel_306/1951040339.py:1: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  r_ppl = to_snakecase(pd.read_csv(f"{GCS_PATH}{replica_ppl}"))


replica people dataframe sample

In [134]:
r_ppl.sample()

,person_id,work_st_2020,household_income,resident_type,language,available_vehicles,race_ethnicity,education,work_bgrp_2020,wfh,home_cty_2020,home_trct_2020,tenure,industry,work_trct_2020,school_grade_attending,building_type,commute_mode,employment_status,work_cty_2020,individual_income,home_st_2020,sex,household_size,home_bgrp_2020,household_id,age
183579,14190272423718015740,Does not have work/school location,80134.0,core,english,three_plus,white_not_hispanic_or_latino,high_school,Does not have work/school location,unemployed_under_16_not_in_labor_force,San Joaquin,"41.04 (San Joaquin, CA)",owner,not_working,Does not have work/school location,not_attending_school,single_family,other_travel_mode,not_in_labor_force,Does not have work/school location,12629.0,California,female,3.0,"2 (Tract 41.04, San Joaquin, CA)",5519997059193984488,58.0


In [145]:
display(HTML("<h4><strong>Number of Travelers by Resident Type: Replica People Data</strong></h4>")) 

In [97]:
r_ppl>>group_by(_.resident_type)>>summarize(_unique_ids = _.person_id.nunique())

,resident_type,_unique_ids
0,core,278158
1,visitor,23034


In [146]:
display(HTML("<h4><strong>Number of Travelers by Resident Type: Replica Trips Data</strong></h4>")) 

In [147]:
r_trips>>group_by(_.trip_taker_resident_type)>>summarize(_unique_ids = _.trip_taker_person_id.nunique())

,trip_taker_resident_type,_unique_ids
0,core,282789
1,visitor,26806
2,NaN,1


In [153]:
display(HTML("<h4><strong>Traveler Demographics: Replica People Data</strong></h4>")) 
display(HTML("Fitered for Core Residents")) 
(r_ppl
 >>filter(_.home_bgrp_2020!="Visitor (no home location)")
 >>group_by(_.commute_mode, _.sex)
 >>summarize(
     n_ppl = _.person_id.nunique(),
     avg_h_income = _.household_income.mean(),
     avg_p_income = _.individual_income.mean(),
     avg_age = _.age.mean())
)

,commute_mode,sex,n_ppl,avg_h_income,avg_p_income,avg_age
0,auto_passenger,female,10908,175717.982765,70255.626054,42.234507
1,auto_passenger,male,11605,177241.394916,82072.291685,42.185523
2,biking,female,234,248565.141026,82592.418803,36.085470
3,biking,male,527,219563.083491,120152.859583,38.426945
4,other_travel_mode,female,36176,126129.721003,21413.682939,55.161599
5,other_travel_mode,male,26290,116775.549106,33563.078813,54.328262
6,private_auto,female,45312,186499.659406,76492.005341,42.626302
7,private_auto,male,59762,190643.201265,100712.386918,42.872093
8,public_transit,female,22008,193540.921347,84074.072110,41.921937
9,public_transit,male,26480,207838.001699,125445.942485,42.259290


In [152]:
display(HTML("<h4><strong>Traveler Demographics: Replica Trips Data</strong></h4>")) 
display(HTML("Fitered for Core Residents")) 
(r_trips
 >>filter(_.trip_taker_home_bgrp_2020!="Visitor (no home location)")
 >>group_by(_.primary_mode, _.trip_taker_sex)
 >>summarize(
     n_ppl = _.trip_taker_person_id.nunique(),
     avg_h_income = _.trip_taker_household_income.mean(),
     avg_p_income = _.trip_taker_individual_income.mean(),
     avg_age = _.trip_taker_age.mean())
)
     

,primary_mode,trip_taker_sex,n_ppl,avg_h_income,avg_p_income,avg_age
0,auto_passenger,female,99350,173666.662228,65104.398490,45.064633
1,auto_passenger,male,105359,187590.985442,97296.509379,43.991868
2,commercial,NaN,1,NaN,NaN,NaN
3,on_demand_auto,female,9051,186226.833189,79990.395102,42.900399
4,on_demand_auto,male,10159,202467.780725,111915.392968,42.436605
5,private_auto,female,68268,174867.937251,62299.064752,45.316785
6,private_auto,male,75223,182301.952228,90322.373466,44.265320
7,public_transit,female,2256,186720.499643,82496.554922,39.104494
8,public_transit,male,2535,205334.441232,114041.914833,38.312382


In [156]:
replica_people_demographics = (r_ppl
 >>group_by(_.commute_mode, _.sex)
 >>summarize(
     n_ppl = _.person_id.nunique(),
     avg_h_income = _.household_income.mean(),
     avg_p_income = _.individual_income.mean(),
     avg_age = _.age.mean())
)

In [157]:
replica_people_demographics

,commute_mode,sex,n_ppl,avg_h_income,avg_p_income,avg_age
0,auto_passenger,female,10908,175717.982765,70255.626054,42.234507
1,auto_passenger,male,11605,177241.394916,82072.291685,42.185523
2,biking,female,234,248565.141026,82592.418803,36.085470
3,biking,male,527,219563.083491,120152.859583,38.426945
4,other_travel_mode,female,36176,126129.721003,21413.682939,55.161599
5,other_travel_mode,male,26290,116775.549106,33563.078813,54.328262
6,private_auto,female,45312,186499.659406,76492.005341,42.626302
7,private_auto,male,59762,190643.201265,100712.386918,42.872093
8,public_transit,female,22008,193540.921347,84074.072110,41.921937
9,public_transit,male,26480,207838.001699,125445.942485,42.259290


In [175]:
chart = (
        alt.Chart(replica_people_demographics)
        .mark_bar()
        .encode(
            x=alt.X("commute_mode", title = "Mode"),
            y=alt.Y("n_ppl", title = "Number of People"),
            color=alt.Color("commute_mode", scale=alt.Scale(range = cp.CALITP_DIVERGING_COLORS)),
            tooltip=replica_people_demographics.columns.tolist())
      .properties(title = "Trips by Resident Type",
    width=800,
    height=500)
        )
chart 

alt.Chart(...)

### Read in Streetlight Data: 2022

In [100]:
streetlight = "streetlight_bay_bridge_corridor_study_corridor_study.csv"

In [101]:
sl_data = to_snakecase(pd.read_csv(f"{GCS_PATH}{streetlight}"))

In [136]:
display(HTML("<h4><strong>Streetlight Data Sample</strong></h4>")) 


In [137]:
sl_data.sample()

,data_periods,mode_of_travel,zone_id,zone_name,road_classification,line_zone_length__miles_,zone_is_pass_through,zone_direction__degrees_,zone_cardinal_direction,zone_is_bi_direction,day_type,day_part,average_daily_segment_traffic__stl_volume_,avg_segment_speed__mph_,avg_segment_travel_time__sec_,free_flow_speed__mph_,vehicle_miles_of_travel__stl_volume_,travel_time_index,congested_segment,_85th_speed_percentile
876,"Jan 01, 2022 - Dec 31, 2022",All Vehicles CVD Plus - StL All Vehicles Volume,1114495634,I 80 / 395419317 / 1,Motorway,0.213,yes,82,EAST,no,2: Weekend Day (Sa-Su),06: 4am (4am-5am),864,68,11,69.063,184.44,1.01,False,75


#### Data Explorations: Streetlight

In [177]:
display(HTML("<strong>Length of the Streetlight Data</strong>")) 

len(sl_data)

1710

In [174]:
# sl_data>>count(_.zone_id, _.zone_name)

In [105]:
sl_data>>filter(_.zone_id==1133975975)>>count(_.day_part)

,day_part,n
0,00: All Day (12am-12am),3
1,01: 12am (12am-1am),3
2,02: Early AM (12am-6am),3
3,03: 1am (1am-2am),3
4,04: 2am (2am-3am),3
5,05: 3am (3am-4am),3
6,06: 4am (4am-5am),3
7,07: 5am (5am-6am),3
8,08: 6am (6am-7am),3
9,09: Peak AM (6am-10am),3


* The data here shows that each segment is broken out into hour time slots and then also aggregated to peak time periods. moving forward we will look at the `All Day` for `day_part` and `All Days` for `day_type`

In [106]:
sl_data_single_day = sl_data>>filter(_.day_part == '00: All Day (12am-12am)', _.day_type == '0: All Days (M-Su)')

In [172]:
# len(sl_data_single_day)

In [173]:
# sl_data_single_day>>count(_.zone_id, _.zone_name)

In [171]:
# sl_data_single_day>>filter(_.zone_id==1133975975)

In [110]:
sl_data_single_day.sample(3)

,data_periods,mode_of_travel,zone_id,zone_name,road_classification,line_zone_length__miles_,zone_is_pass_through,zone_direction__degrees_,zone_cardinal_direction,zone_is_bi_direction,day_type,day_part,average_daily_segment_traffic__stl_volume_,avg_segment_speed__mph_,avg_segment_travel_time__sec_,free_flow_speed__mph_,vehicle_miles_of_travel__stl_volume_,travel_time_index,congested_segment,_85th_speed_percentile
270,"Jan 01, 2022 - Dec 31, 2022",All Vehicles CVD Plus - StL All Vehicles Volume,1032819756,San Francisco – Oakland Bay Bridge / 11415208 / 1,Motorway,0.109,yes,220,WEST,no,0: All Days (M-Su),00: All Day (12am-12am),132245,49,10,68.596,14450.00,1.41,True,67
450,"Jan 01, 2022 - Dec 31, 2022",All Vehicles CVD Plus - StL All Vehicles Volume,1050419851,San Francisco – Oakland Bay Bridge / 179235221 / 1,Motorway,0.004,yes,38,EAST,no,0: All Days (M-Su),00: All Day (12am-12am),161170,53,2,67.840,695.17,1.29,True,67
0,"Jan 01, 2022 - Dec 31, 2022",All Vehicles CVD Plus - StL All Vehicles Volume,1005898592,San Francisco – Oakland Bay Bridge / 8921938 / 1,Motorway,1.985,yes,220,WEST,no,0: All Days (M-Su),00: All Day (12am-12am),108456,44,179,66.336,215328.00,1.50,True,63


In [170]:
# sl_data_single_day>>group_by(_.zone_cardinal_direction, _.day_type, _.day_part)>>summarize(avg_speed= _.avg_segment_speed__mph_.mean(),
#                                                                                           avg_travel_time = _.avg_segment_travel_time__sec_.mean(),
#                                                                                           avg_volume = _.average_daily_segment_traffic__stl_volume_.mean(),
#                                                                                           sum_volume = _.average_daily_segment_traffic__stl_volume_.sum())

In [112]:
sl_peaks = (sl_data>>filter(
                            _.day_part != "01: 12am (12am-1am)",
                            _.day_part != "03: 1am (1am-2am)",
                            _.day_part != "04: 2am (2am-3am)",
                            _.day_part != "05: 3am (3am-4am)",
                            _.day_part != "06: 4am (4am-5am)",
                            _.day_part != "07: 5am (5am-6am)",
                            _.day_part != "08: 6am (6am-7am)",
                            _.day_part != "10: 7am (7am-8am)",
                            _.day_part != "11: 8am (8am-9am)",
                            _.day_part != "12: 9am (9am-10am)",
                            _.day_part != "13: 10am (10am-11am)",
                            _.day_part != "15: 11am (11am-12noon)",
                            _.day_part != "16: 12pm (12noon-1pm)",
                            _.day_part != "17: 1pm (1pm-2pm)",
                            _.day_part != "18: 2pm (2pm-3pm)",
                            _.day_part != "19: 3pm (3pm-4pm)",
                            _.day_part != "20: 4pm (4pm-5pm)",
                            _.day_part != "22: 5pm (5pm-6pm)",
                            _.day_part != "23: 6pm (6pm-7pm)",
                            _.day_part != "24: 7pm (7pm-8pm)",
                            _.day_part != "25: 8pm (8pm-9pm)",
                            _.day_part != "27: 9pm (9pm-10pm)",
                            _.day_part != "28: 10pm (10pm-11pm)",
                            _.day_part !="29: 11pm (11pm-12am)"))

In [113]:
sl_hourly = (sl_data>>filter(_.day_part != "00: All Day (12am-12am)",
                            _.day_part != "02: Early AM (12am-6am)",
                            _.day_part != "09: Peak AM (6am-10am)",
                            _.day_part != "14: Mid-Day (10am-4pm)",
                            _.day_part != "21: Peak PM (4pm-8pm)", _.day_part !="26: Late PM (8pm-12am)"))

In [114]:
sl_peaks_agg = sl_peaks>>group_by(_.zone_cardinal_direction,
    _.day_type, _.day_part)>>summarize(avg_speed= _.avg_segment_speed__mph_.mean(),
                                                                                          avg_travel_time = _.avg_segment_travel_time__sec_.mean(),
                                                                                          avg_volume = _.average_daily_segment_traffic__stl_volume_.mean(),
                                                                                          sum_volume = _.average_daily_segment_traffic__stl_volume_.sum())

In [169]:
# sl_peaks_agg.sample()

In [166]:
chart = (
        alt.Chart(sl_peaks_agg)
        .mark_bar()
        .encode(
            x=alt.X("day_part"),
            y=alt.Y("avg_speed"),
            color=alt.Color("zone_cardinal_direction", scale=alt.Scale(range = cp.CALITP_DIVERGING_COLORS,
            domain=sl_peaks_agg["zone_cardinal_direction"].unique().tolist())
            ),
            tooltip=sl_peaks_agg.columns.tolist(),
            )
      .properties(title = "Average Speed by Direction by Day Part",
    width=800,
    height=300)
        )
chart 

alt.Chart(...)

In [168]:
chart = (
        alt.Chart(sl_peaks_agg)
        .mark_bar()
        .encode(
            x=alt.X("day_part"),
            y=alt.Y("sum_volume"),
            color=alt.Color("zone_cardinal_direction", scale=alt.Scale(range = cp.CALITP_DIVERGING_COLORS,
            domain=sl_peaks_agg["zone_cardinal_direction"].unique().tolist())
            ),
            tooltip=sl_peaks_agg.columns.tolist(),
            )
      .properties(title = "Average Volume by Direction by Day Part",
    width=800,
    height=300)
        )
chart 

alt.Chart(...)

In [118]:
sl_hourly_agg = sl_hourly>>filter(_.day_type=="0: All Days (M-Su)")>>group_by(_.zone_cardinal_direction,
    _.day_type, _.day_part)>>summarize(avg_speed= _.avg_segment_speed__mph_.mean(),
                                                                                          avg_travel_time = _.avg_segment_travel_time__sec_.mean(),
                                                                                          avg_volume = _.average_daily_segment_traffic__stl_volume_.mean(),
                                                                                          sum_volume = _.average_daily_segment_traffic__stl_volume_.sum())

In [161]:
# sl_hourly_agg.sample()

In [164]:
chart = (
        alt.Chart(sl_hourly_agg)
        .mark_bar()
        .encode(
            x=alt.X("day_part"),
            y=alt.Y("avg_speed"),
            color=alt.Color("zone_cardinal_direction", scale=alt.Scale(range = cp.CALITP_DIVERGING_COLORS,
            domain=sl_hourly_agg["zone_cardinal_direction"].unique().tolist())
            ),
            tooltip=sl_hourly_agg.columns.tolist(),
            )
      .properties(title = "Average Speed by Hour", 
    width=800,
    height=300)
        )
chart 

alt.Chart(...)

In [167]:
chart = (
        alt.Chart(sl_hourly_agg)
        .mark_bar()
        .encode(
            x=alt.X("day_part"),
            y=alt.Y("sum_volume"),
            color=alt.Color("zone_cardinal_direction", scale=alt.Scale(range = cp.CALITP_DIVERGING_COLORS,
            domain=sl_hourly_agg["zone_cardinal_direction"].unique().tolist())
            ),
            tooltip=sl_hourly_agg.columns.tolist(),
            )
      .properties(title = "Total Volume by Hour",
    width=800,
    height=300)
        )
chart 

alt.Chart(...)